### Metaclasses vs Class Decorators

As we have seen, class decorators can achieve a lot of the metaprogramming goals we might have.

But there is one area where they fall short of metaclasses - inheritance.

Metaclasses are carried through inheritance, whereas decorators are not.

Let's go back to the previous class decorator example we had (and I'll use the original one to keep the code simple):

In [1]:
from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner    

def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

And as we saw, we can decorate a class with it:

In [2]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'

In [3]:
Person('Alex', 10).greet()

We could do this with a metaclass too:

In [4]:
class ClassLogger(type):
    def __new__(mcls, name, bases, class_dict):
        new_cls = super().__new__(mcls, name, bases, class_dict)
        for key, obj in vars(new_cls).items():
            if callable(obj):
                setattr(new_cls, key, func_logger(obj))
        return new_cls        

In [5]:
class Person(metaclass=ClassLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'

In [6]:
p = Person('John', 78).greet()

log: Person.__init__((<__main__.Person object at 0x7f9be0ce18d0>, 'John', 78), {}) = None
log: Person.greet((<__main__.Person object at 0x7f9be0ce18d0>,), {}) = Hello, my name is John and I am 78


So, why not just use a class decorator?

Now let's see how inheritance works with both those methods.

Let's do the decorator approach first:

In [7]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet


Now let's inherit `Person`:

In [8]:
class Student(Person):
    def __init__(self, name, age, student_number):
        super().__init__(name, age)
        self.student_number = student_number
        
    def study(self):
        return f'{self.name} studies...'

In [9]:
s = Student('Alex', 19, 'abcdefg')

log: Person.__init__((<__main__.Student object at 0x7f9be0cec790>, 'Alex', 19), {}) = None


So first off, you can see that the print worked, but only for the `__init__` in the `Person` class, no logs were generated for the `__init__` in the `Student` class.

By the same token, we don't get logging on the `study` method:

In [10]:
s.study()

'Alex studies...'

So we would need to remember to decorate the `Student` class as well:

In [11]:
@class_logger
class Student(Person):
    def __init__(self, name, age, student_number):
        super().__init__(name, age)
        self.student_number = student_number
        
    def study(self):
        return f'{self.name} studies...'

decorating: <class '__main__.Student'> __init__
decorating: <class '__main__.Student'> study


In [12]:
s = Student('Alex', 19, 'abcdefg')

log: Person.__init__((<__main__.Student object at 0x7f9be0ce1090>, 'Alex', 19), {}) = None
log: Student.__init__((<__main__.Student object at 0x7f9be0ce1090>, 'Alex', 19, 'abcdefg'), {}) = None


In [13]:
s.greet()

log: Person.greet((<__main__.Student object at 0x7f9be0ce1090>,), {}) = Hello, my name is Alex and I am 19


'Hello, my name is Alex and I am 19'

In [14]:
s.study()

log: Student.study((<__main__.Student object at 0x7f9be0ce1090>,), {}) = Alex studies...


'Alex studies...'

So, we just have to remember to decorate **every** subclass as well.

But if we use a metaclass, watch what happens when inherit:

In [15]:
class Person(metaclass=ClassLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'
    
class Student(Person):
    def __init__(self, name, age, student_number):
        super().__init__(name, age)
        self.student_number = student_number
        
    def study(self):
        return f'{self.name} studies...'

In [16]:
s = Student('Alex', 19, 'abcdefg')

log: Person.__init__((<__main__.Student object at 0x7f9be0cff210>, 'Alex', 19), {}) = None
log: Student.__init__((<__main__.Student object at 0x7f9be0cff210>, 'Alex', 19, 'abcdefg'), {}) = None


In [17]:
s.study()

log: Student.study((<__main__.Student object at 0x7f9be0cff210>,), {}) = Alex studies...


'Alex studies...'

This works because `Student` inherits from `Person`, and since `Person` uses a metaclass for the creation, this follows down to the `Student` class as well.

In [18]:
type(Person)

__main__.ClassLogger

In [19]:
type(Student)

__main__.ClassLogger

As you can see the type of both the parent and the subclass is `ClassLogger` even though we did not explicitly state that `Student` shouls use the metaclass for creation.

It happened automatically because we did not have a `__new__` method in the `Student` class, so the parent's `__new__` was essentially used, and that one uses the metaclass.

We can see this more explicitly this way:

In [20]:
class Student(Person):
    def __new__(cls, name, age, student_number):
        return super().__new__(cls)
    
    def __init__(self, name, age, student_number):
        super().__init__(name, age)
        self.student_number = student_number
        
    def study(self):
        return f'{self.name} studies...'

In [21]:
s = Student('Alex', 19, 'ABC')

log: Person.__init__((<__main__.Student object at 0x7f9be0d041d0>, 'Alex', 19), {}) = None
log: Student.__init__((<__main__.Student object at 0x7f9be0d041d0>, 'Alex', 19, 'ABC'), {}) = None


In [22]:
s.study()

log: Student.study((<__main__.Student object at 0x7f9be0d041d0>,), {}) = Alex studies...


'Alex studies...'

One of the disadvantages of metaclasses vs class decorators is that only a "single" metaclass can be used. (Actually it's a bit more subtle than that, we can use a different metaclass in for a subclass if the metclass is a subclass of the parent's metaclass - we'll cover this point again when we look at multiple inheritance.)

In [26]:
class Metaclass1(type):
    pass

class Metaclass2(type):
    pass

In [27]:
class Person(metaclass=Metaclass1):
    pass

In [28]:
class Student(Person, metaclass=Metaclass2):
    pass

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

As you can see we cannot specify a custom metaclass for `Student` because that would conflict with the class it is inheriting from.

An exception is if we inherit from a parent who has `type` as its metaclass:

In [29]:
class Person:
    pass

class Student(Person, metaclass=Metaclass1):
    pass

In [30]:
p = Person()
s = Student()

In [31]:
type(Person), type(Student)

(type, __main__.Metaclass1)

It can also cause problems in multiple inheritance.

We haven't covered multiple inheritance yet, but let me show you the issue at least:

In [32]:
class Class1(metaclass=Metaclass1):
    pass

class Class2(metaclass=Metaclass2):
    pass

Here we have created two classes that use different custom metaclasses.

If we try to create a new class that inherits from both:

In [33]:
class MultiClass(Class1, Class2):
    pass

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

Again, if one of the base classes is `type` and the other is a custom metaclass, then this is allowed (this is because `Metaclass1` is itself a subclass of `type`:

In [36]:
class Class1(metaclass=type):
    pass

class Class2(metaclass=Metaclass1):
    pass

In [37]:
class MultiClass(Class1, Class2):
    pass

On the other hand we can stack decorators as much as we want (we just have to be careful with the order in which we stack them sometimes).